In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# IMPORTING THE REQUIRED LIBRARIES
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import nltk
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()
from nltk import FreqDist
import string
import spacy
nlp= spacy.load("en")

In [ ]:
# IMPORTING THE DATA
data = pd.read_csv("../input/fake.csv")

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
data['language'].unique()

In [ ]:
data.groupby(["type"]).size().plot(kind="barh")

In [ ]:
txt= data["title"][1009]

In [ ]:
doc = nlp(txt)

In [ ]:
doc

In [ ]:
txt

In [ ]:
doc = nlp(txt)    
olist = []
for token in doc:
    l = [token.text,
        token.idx,
        token.lemma_,
        token.is_punct,
        token.is_space,
        token.shape_,
        token.pos_,
        token.tag_]
    olist.append(l)
    
odf = pd.DataFrame(olist)
odf.columns= ["Text", "StartIndex", "Lemma", "IsPunctuation", "IsSpace", "WordShape", "PartOfSpeech", "POSTag"]
odf

In [ ]:
doc =nlp(txt)
olist= []
for ent in doc.ents:
    olist.append([ent.text,ent.label_])

In [ ]:
odf = pd.DataFrame(olist)
odf.columns = ["Text", "EntityType"]
odf

 Performing noum phrase chunking using spacy
   * Phrase chunking means noun plus the words describing the noun

In [ ]:
txt= data["title"][2012]

In [ ]:
doc =nlp(txt)
olist= []
for chunk in doc.noun_chunks:
    olist.append([chunk.text, chunk.label_,chunk.root.text])
odf =pd.DataFrame(olist)
odf.columns=["NounPhrase","Label","RootWord"]
odf

## Dependency Parser

A dependency parser analyzes the grammatical structure of a sentence, establishing relationships between "head" words and words which modify those heads - Stanford NLP

Spacy can be used to create these dependency parsers which can be used in a variety of tasks.

In [ ]:
doc = nlp(data["title"][1009])
olist = []
for token in doc:
    olist.append([token.text, token.dep_, token.head.text, token.head.pos_,
          [child for child in token.children]])
odf = pd.DataFrame(olist)
odf.columns = ["Text", "Dep", "Head text", "Head POS", "Children"]
odf

In [ ]:
"""from scipy import spatial
nlp = spacy.load('en_core_web_lg')
cosine_similarity = lambda x,y: 1-spatial.distance.cosine(x,y)
queen = nlp.vocab["Queen"].vector
compound_simi = []
for word in nlp.vocab:
    if not word.has_vector:
        continue
    simi = cosine_similarity(queen, word.vector)
    compound_simi.append((word, simi))

compound_simi = sorted(compound_simi, key=lambda item: -item[1])
print([w[0].text for w in compound_simi[:10]])"""

In [ ]:
data.loc[1,["title","text"]]

In [ ]:
data.title.fillna("", inplace=True)
data.text.fillna("", inplace=True)
all_text = data.title.str.cat(data.text, sep=' ')

In [ ]:
words= nltk.word_tokenize(" ".join(all_text.tolist()))

In [ ]:
from nltk.corpus import stopwords
import string
stop_w= stopwords.words("english")
#cleanwords = [i for i in words if i not in stop_w and i.isalpha() and len(i) > 2]

In [ ]:
from wordcloud import WordCloud, STOPWORDS
wordcloud2 =WordCloud(
                    stopwords= STOPWORDS,
                    background_color= "white",
                    width= 1200,
                    height= 1000,
                    ).generate(" ".join(cleanwords))
plt.imshow(wordcloud2)
plt.axis("off")
plt.show()

In [ ]:
## WORKIGN WITH BIGRAMS
bigrams = nltk.bigrams(cleanwords)

In [ ]:
from collections import Counter
counter =Counter(bigrams)
print(counter.most_common(10))

In [ ]:
data.head()

In [ ]:
data.groupby("language").size().plot("barh")

In [ ]:
Counter(data["language"])

### Filtering out the most uncommon words out from the whole corpus

In [ ]:
"""def filter_words(word,top_k=1000,bottom_k=1200):
    fdist= FreqDist(words)
    top_k_words= fdist.most_common(top_k)
    bottom_k_words= fdist.most_common(bottom_k)
    top_f= set(zip(*top_k_words))
    bottom_f= set(zip(*bottom_k_words))
    return(top_f,bottom_f)"""
top_k=10000
bottom_k=15000
fdist= FreqDist(words)
top_k_words= fdist.most_common(top_k)
bottom_k_words= fdist.most_common(bottom_k)
top_f= set(zip(*top_k_words))
bottom_f= set(zip(*bottom_k_words))


### Cleaning out the text

In [ ]:
# FUNCTION FOR CLEANING OUT THE DATAFRAME'S COLUMNS WORDS
def dataclean(dff,words=words):
    #top_w, bottom_w=filter_words(words)
    table =str.maketrans("","",string.punctuation)
    words= dff.split(" ")
    desc= [lem.lemmatize(w.lower()) for w in words if not w in stop_w]
    #desc= [w for w in desc if w not in bottom_w]
    #desc= [w for w in desc if w not in top_w]
    desc= [w.translate(table) for w in desc]
    desc= [w for w in desc if w.isalpha()]
    desc= [w for w in desc if len(w)>2]
    desc= " ".join(desc)
    return(desc)

In [ ]:
x= data[data["language"]=="english"]["text"].apply(dataclean)

In [ ]:
y= data[data["language"]=="english"]["title"].apply(dataclean)

In [ ]:
ddf=x.str.cat(y,sep=" ")

In [ ]:
sns.countplot(data[data["language"]=="english"]["type"])

In [ ]:
# Preparing the TfidfVectorizer of the simpliified dataframe
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=8000)
X = vectorizer.fit_transform(ddf)
feat= vectorizer.get_feature_names()
X= X.toarray()
ddf= pd.DataFrame(X, columns=feat)

In [ ]:
# FORMING THE CLUSTER USING THE GENERATED ARRAY IN TFIDF USING KMEANS ALGORITHM
from sklearn.cluster import KMeans
import numpy as np
kmeans = KMeans(n_clusters=2, random_state=0).fit(ddf)
kmeans.labels_
kmeans.cluster_centers_

In [ ]:
# PRDICTING THE RESULT
pred= kmeans.fit_predict(ddf.iloc[200:4000,:])

In [ ]:
## REDUCING THE DIMENSIONS OF THE DATASET FOR VISUALIZATION
import numpy as np
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2).fit_transform(ddf)
X_embedded.shape

In [ ]:
# MAKING THE PLOT OF THE SCATTERED DATA
plt.scatter(X_embedded[pred==0,0],X_embedded[pred==0,1],color="r")
plt.scatter(X_embedded[pred==1,0],X_embedded[pred==1,1],color="b")